# Scraping and Clustering Toronto with Python
# Part 2 - Locating Neighborhoods

## Introduction

In this Part 2 of 3 of the series, we will locate the coordinates of neighborhoods in Toronto using the Nominatim library, save into the dataframe for furtherprocessing in Part 3.

### Import the necessary libraries

In [52]:
import pandas as pd
from geopy.geocoders import Nominatim

A function to get coordinates using the Nominatim library. Checking on some neighborhoods might throw an AttributeError. If it happens, check the coordinates using the borough name instead.

In [53]:
def get_coordinates(address):
    geolocator = Nominatim(user_agent = "toronto_explorer")

    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    
    except AttributeError:
        latitude = None
        longitude = None

    return latitude, longitude

Make sure the function works.

In [54]:
get_coordinates('Toronto')

(43.6534817, -79.3839347)

## Load the dataframe

In [55]:
df_toronto = pd.read_csv('toronto1.csv', index_col = 0)
df_toronto

,PostalCode,Borough,Neighborhood
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
M8X,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,M4Y,Downtown Toronto,Church and Wellesley
M7Y,M7Y,East Toronto,East Toronto
M8Y,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [56]:
df_toronto.shape

(103, 3)

## Get coordinates and store into dataframe

In [57]:
latitudes = []
longitudes = []

for borough, neigh_data in zip(df_toronto['Borough'], df_toronto['Neighborhood']):
    # Some rows consist of multiple neighborhoods. Use the first neighborhood.
    neigh = neigh_data.split(', ')[0]
    neigh += ', Toronto'
    
    coordinates = get_coordinates(neigh)
    
    if coordinates[0] == None or coordinates[1] == None: # Coordinates not found for neighborhood
        print()
        print('Warning: Coordinates cannot be found for neighborhood {}. Trying borough.'.format(neigh))
        
        coordinates = get_coordinates(borough + ', Toronto') # Use borough instead
        
        print('Coordinates found for borough {} - {}'.format(borough, coordinates))
        print()
        
    else:
        print('Coordinates found for neighborhood {} - {}'.format(neigh, coordinates))
    
    latitudes.append(coordinates[0])
    longitudes.append(coordinates[1])

df_toronto['Latitude'] = latitudes
df_toronto['Longitude'] = longitudes

Coordinates found for neighborhood Parkwoods, Toronto - (43.7587999, -79.3201966)
Coordinates found for neighborhood Victoria Village, Toronto - (43.732658, -79.3111892)
Coordinates found for neighborhood Regent Park, Toronto - (43.6607056, -79.3604569)
Coordinates found for neighborhood Lawrence Manor, Toronto - (43.7220788, -79.4375067)
Coordinates found for neighborhood Queen's Park, Toronto - (43.659659, -79.3903399)
Coordinates found for neighborhood Islington Avenue, Toronto - (43.6794838, -79.5389092)
Coordinates found for neighborhood Malvern, Toronto - (43.8091955, -79.2217008)
Coordinates found for neighborhood Don Mills, Toronto - (43.775347, -79.3459439)
Coordinates found for neighborhood Parkview Hill, Toronto - (43.7062977, -79.3219073)
Coordinates found for neighborhood Garden District, Toronto - (43.6564995, -79.3771141)
Coordinates found for neighborhood Glencairn, Toronto - (43.7087117, -79.4406853)
Coordinates found for neighborhood West Deane Park, Toronto - (43.663

Coordinates found for neighborhood Alderwood, Toronto - (43.6017173, -79.5452325)
Coordinates found for neighborhood Northwest, Toronto - (43.6534817, -79.3839347)
Coordinates found for neighborhood Upper Rouge, Toronto - (43.8049304, -79.1658374)
Coordinates found for neighborhood St. James Town, Toronto - (43.6694032, -79.3727041)
Coordinates found for neighborhood First Canadian Place, Toronto - (43.6487681, -79.38169179283025)
Coordinates found for neighborhood The Kingsway, Toronto - (43.6473811, -79.5113328)
Coordinates found for neighborhood Church and Wellesley, Toronto - (43.6655242, -79.3838011)
Coordinates found for neighborhood East Toronto, Toronto - (43.6534817, -79.3839347)
Coordinates found for neighborhood Old Mill South, Toronto - (43.649826, -79.4943338)
Coordinates found for neighborhood Mimico NW, Toronto - (43.6166773, -79.4968048)


Check to ensure no coordinates are missing.

In [58]:
df_toronto[(df_toronto['Latitude'].isnull()) | (df_toronto['Longitude'].isnull())]

,PostalCode,Borough,Neighborhood,Latitude,Longitude


### The resulting dataframe

In [59]:
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
M3A,M3A,North York,Parkwoods,43.758800,-79.320197
M4A,M4A,North York,Victoria Village,43.732658,-79.311189
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360457
M6A,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722079,-79.437507
M7A,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.659659,-79.390340
...,...,...,...,...,...
M8X,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.647381,-79.511333
M4Y,M4Y,Downtown Toronto,Church and Wellesley,43.665524,-79.383801
M7Y,M7Y,East Toronto,East Toronto,43.653482,-79.383935
M8Y,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.649826,-79.494334


In [60]:
df_toronto.shape

(103, 5)

Write dataframe into CSV file to be used in Part 3.

In [61]:
df_toronto.to_csv('toronto2.csv')